In [177]:
import numpy as np
import pandas as pd

In [178]:
excel_file_path_sfdc_version = r'C:\Users\phsheari\Documents\SFDC Data\PIPELINE UPDATE\SFDC_Trackerfile.pkl'

In [179]:
df = pd.read_pickle(excel_file_path_sfdc_version)

In [180]:
readouts = df[['Request ID','Customer Name','Date Readout Done']]

In [181]:
#A function to stringify the various possible elements in the dataframe with a float data-type
def stringify_element(m):
    a = pd.NA
    if pd.isnull(m):  # if did is null or nan
        return pd.NA
    elif type(m) == np.float64:
        return str(int(m))
    elif type(m) == float:
        return str(int(m))
    elif type(m) == np.int64:
        return str(m)
    elif type(m) == int:
        return str(m)
    elif type(m) == str:
        try:
            a = m.replace('"','').replace('&',',').replace('(','').replace(')','').replace(':',',').replace(' ',',').replace('\n',',').replace(',,',',').strip().split(',') 
            return a
        except:
            import pdb; pdb.set_trace()
    else:
        return m

In [182]:
guidf = df[['Request ID','GUID']]

In [183]:
guidf['GUID'] = guidf['GUID'].apply(stringify_element)

C:\Users\phsheari\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [184]:
guidf.set_index('Request ID', inplace=True)

In [185]:
guidf = guidf.explode('GUID')

In [186]:
guidf.reset_index(inplace=True)

In [187]:
guidf.dropna(inplace=True)

In [188]:
guidlist = []
for i,guid in guidf['GUID'].items():
    if isinstance(guid,list):
        for e in guid:
            if (pd.notnull(e) or e != ''):
                if e.isdigit():
                    guidlist.append((i,e))
    else:
        guidlist.append((i,guid))

In [189]:
guidf['BADGUID'] = guidf['GUID'].str.isalpha()

In [190]:
guidf['BADLENGTH'] = guidf['GUID'].apply(lambda x: len(x)> 10)

In [191]:
guidf.drop(guidf.loc[guidf['BADGUID']==True].index, inplace=True)

In [192]:
guidf.drop(guidf.loc[guidf['BADLENGTH']==True].index, inplace=True)

In [193]:
guidf.drop(guidf.loc[~(guidf['GUID'].str.isdigit())].index, inplace=True)

In [194]:
guidf.drop(columns = ['BADGUID','BADLENGTH'], inplace=True)

In [195]:
guidf['GUID'] = guidf['GUID'].astype(int)

In [199]:
guidf

,Request ID,GUID
0,201820181129,6725524
1,201820181167,180320
2,201820181195,153753906
3,201820181195,2689130
4,201820181217,8109463
...,...,...
26195,201820187130,61814
26196,201820187131,14650
26197,201820187132,3096947
26198,201820187133,10173


In [196]:
newdf = pd.merge(guidf, readouts, on='Request ID').dropna()

In [197]:
newdf

,Request ID,GUID,Date Readout Done
2,201820181195,153753906,2020-02-27
3,201820181195,2689130,2020-02-27
4,201820181217,8109463,2020-02-03
5,201820181227,212964998,2019-12-20
6,201820181257,3773463,2019-12-20
...,...,...,...
24883,201820187076,9616260,2020-08-21
24884,201820187077,8711890,2020-08-21
24896,201820187089,4267384,2020-08-19
24903,201820187096,31676772,2020-08-19


In [198]:
newdf.loc[newdf['GUID']==8109463,'Date Readout Done'].values[0]

datetime.date(2020, 2, 3)